📒 Add paths and import required modules


In [1]:
# Add root and utils to sys.path
import sys
import os

sys.path.append(os.path.abspath(".."))           # for model.py
sys.path.append(os.path.abspath("../utils"))     # for utils.py, logger.py

# Imports
import torch
import numpy as np
from model import ChordConditionedMelodyTransformer
from utils.utils import pitch_to_midi
import matplotlib.pyplot as plt


📒 Define the model configuration
Make sure this matches your hparams.yaml.

In [2]:
# Model configuration (should match training)
model_config = {
    "num_pitch": 50,
    "frame_per_bar": 16,
    "num_bars": 8,
    "chord_emb_size": 128,
    "pitch_emb_size": 256,
    "hidden_dim": 512,
    "key_dim": 512,
    "value_dim": 512,
    "num_layers": 8,
    "num_heads": 16,
    "input_dropout": 0.2,
    "layer_dropout": 0.2,
    "attention_dropout": 0.2
}


📒 Load the trained model from checkpoint


In [3]:
# Load model checkpoint
checkpoint_path = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/results/idx002/model/checkpoint_70.pth.tar"
checkpoint = torch.load(checkpoint_path, map_location="cpu")

model = ChordConditionedMelodyTransformer(**model_config)
model.load_state_dict(checkpoint["model"])
model.eval()

print("✅ Model loaded and ready.")


/tmp/ipykernel_25210/3072443676.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location="cpu")


✅ Model loaded and ready.


📒 Chord helper function

Convert chord names to 12-dimensional binary vectors.

In [4]:
# Simple chord dictionary (expand as needed)
def roman_to_chord_vector(roman):
    chord_templates = {
        'C': [0, 4, 7],
        'F': [5, 9, 0],
        'G': [7, 11, 2],
        'Am': [9, 0, 4],
        'Dm': [2, 5, 9],
        'Em': [4, 7, 11]
    }
    vec = np.zeros(12)
    for p in chord_templates.get(roman, []):
        vec[p % 12] = 1
    return vec


📒 Create a custom chord progression and convert to tensor


In [5]:
# 8 bars × 16 frames per bar = 128 time steps
progression = ['C', 'Am', 'F', 'G', 'C', 'Am', 'F', 'G']
frames_per_bar = model_config["frame_per_bar"]
max_len = model_config["num_bars"] * frames_per_bar

chord_matrix = np.zeros((max_len, 12))
for i, chord in enumerate(progression):
    vec = roman_to_chord_vector(chord)
    chord_matrix[i * frames_per_bar:(i + 1) * frames_per_bar] = vec

chord_tensor = torch.tensor(chord_matrix).unsqueeze(0).float()  # shape: [1, 128, 12]
print(f"✅ chord_tensor shape: {chord_tensor.shape}")



✅ chord_tensor shape: torch.Size([1, 128, 12])


📒 Generate melody from scratch (no prime)


In [6]:
# Empty prime (start from scratch)
prime_rhythm = torch.zeros((1, 0), dtype=torch.long)
prime_pitch = torch.zeros((1, 0), dtype=torch.long)

# Sampling
with torch.no_grad():
    result = model.sampling(prime_rhythm, prime_pitch, chord_tensor, topk=3)

pitch = result["pitch"][0].numpy()
rhythm = result["rhythm"][0].numpy()
print("✅ Melody generated.")


✅ Melody generated.


In [7]:
output_path = "generated_custom.mid"
pitch_to_midi(pitch, chord_matrix, frame_per_bar=frames_per_bar, save_path=output_path)
print(f"✅ MIDI saved as {output_path}")


✅ MIDI saved as generated_custom.mid


In [8]:
import pretty_midi

midi_data = pretty_midi.PrettyMIDI(output_path)
melody_roll = midi_data.instruments[0].get_piano_roll(fs=16)

plt.figure(figsize=(12, 4))
plt.imshow(melody_roll, origin='lower', aspect='auto', cmap='gray_r')
plt.title("Generated Melody - Piano Roll")
plt.xlabel("Time")
plt.ylabel("MIDI Pitch")
plt.savefig("generated_custom_pianoroll.png")
print("✅ Piano roll saved as generated_custom_pianoroll.png")


✅ Piano roll saved as generated_custom_pianoroll.png
